# 예측값 도출

In [1]:
# 라이브러리 호출
import json
import os

import cv2
import keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [2]:
oil_data_path = 'oiltank_dataset/valid_images'
container_data_paht = 'container_dataset/valid_images'

## selective search

In [3]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [4]:
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [5]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [6]:
model = tf.keras.models.load_model('VGG16.h5')

In [12]:
for i in os.listdir(oil_data_path):
    if i[-3:] != 'png':
        os.remove(oil_data_path + f'/{i}')

In [1]:
for e,i in enumerate(os.listdir(oil_data_path)[:10]):
    z += 1
    img = cv2.imread(os.path.join(oil_data_path,i))
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    imout = img.copy()
    for e,result in enumerate(ssresults):
        tmp = []
        if e < 2000:
            x,y,w,h = result
            timage = imout[y:y+h,x:x+w]
            resized = cv2.resize(timage, (224, 224), interpolation = cv2.INTER_AREA)
            img = np.expand_dims(resized, axis=0)
            out= model.predict(img, verbose=0)
            print(out)
            if out[0][0] > 0.99:
                cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
    plt.figure()
    plt.imshow(imout)
    plt.show()



NameError: name 'os' is not defined